In [15]:
%pip install -r requirements.txt

     ---------------------------------------- 0.0/64.5 MB ? eta -:--:--
      --------------------------------------- 1.0/64.5 MB 7.2 MB/s eta 0:00:09
     - -------------------------------------- 2.9/64.5 MB 7.0 MB/s eta 0:00:09
     -- ------------------------------------- 4.2/64.5 MB 7.0 MB/s eta 0:00:09
     --- ------------------------------------ 6.3/64.5 MB 7.6 MB/s eta 0:00:08
     ----- ---------------------------------- 8.1/64.5 MB 8.0 MB/s eta 0:00:08
     ------ --------------------------------- 9.7/64.5 MB 7.8 MB/s eta 0:00:07
     ------- -------------------------------- 11.5/64.5 MB 7.9 MB/s eta 0:00:07
     -------- ------------------------------- 13.1/64.5 MB 7.9 MB/s eta 0:00:07
     -------- ------------------------------- 14.4/64.5 MB 7.8 MB/s eta 0:00:07
     --------- ------------------------------ 16.0/64.5 MB 7.8 MB/s eta 0:00:07
     ----------- ---------------------------- 17.8/64.5 MB 7.9 MB/s eta 0:00:06
     ------------ --------------------------- 19.7/64.

  error: subprocess-exited-with-error
  
  × Building wheel for llama-cpp-python (pyproject.toml) did not run successfully.
  │ exit code: 1
  ╰─> [20 lines of output]
      *** scikit-build-core 0.10.7 using CMake 3.31.1 (wheel)
      *** Configuring CMake...
      2024-12-10 11:13:39,709 - scikit_build_core - WARNING - Can't find a Python library, got libdir=None, ldlibrary=None, multiarch=None, masd=None
      loading initial cache file C:\Users\giuli\AppData\Local\Temp\tmpl10vcwro\build\CMakeInit.txt
      -- Building for: NMake Makefiles
      CMake Error at CMakeLists.txt:3 (project):
        Running
      
         'nmake' '-?'
      
        failed with:
      
         no such file or directory
      
      
      CMake Error: CMAKE_C_COMPILER not set, after EnableLanguage
      CMake Error: CMAKE_CXX_COMPILER not set, after EnableLanguage
      -- Configuring incomplete, errors occurred!
      
      *** CMake configuration failed
      [end of output]
  
  note: This error o

In [ ]:
import chromadb
import os
from chromadb.config import Settings
from sentence_transformers import SentenceTransformer
from PyPDF2 import PdfReader
from huggingface_hub import hf_hub_download
from llama_cpp import Llama

d:\TUIA-NLP\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


# 1. Preparación de los datos

## 1.1 Creación de la base de datos vectorial
Creo todas las funciones necesarias para facilitar el preprocesado de los datos.

In [2]:
embedding_model = SentenceTransformer('all-MiniLM-L6-v2')

In [13]:
client = chromadb.PersistentClient(path=os.path.join(os.getcwd(), 'data', 'vector'))

collection = client.get_or_create_collection(name="arnak")

In [4]:
def read_text_file(file_path: str) -> str:
    with open(file_path, 'r', encoding='utf-8') as file:
        return file.read()

In [5]:
def read_pdf_file(file_path: str) -> str:
    pdf_reader = PdfReader(file_path)
    text = ""
    for page in pdf_reader.pages:
        text += page.extract_text()
    return text

In [6]:
def split_into_chunks(text: str, chunk_size: int = 500) -> list[str]:
    words = text.split()
    return [" ".join(words[i:i+chunk_size]) for i in range(0, len(words), chunk_size)]

In [7]:
def insert_into_chromadb(collection: str, chunks: list[str], embedding_model) -> None:
    for i, chunk in enumerate(chunks):
        embedding = embedding_model.encode(chunk)
        collection.add(
            documents=[chunk],
            metadatas=[{"chunk_id": i}],
            ids=[f"doc_{i}"]
        )

In [8]:
def process_and_store(file_paths: str) -> None:
    for file_path in file_paths:
        if file_path.endswith('.txt'):
            text = read_text_file(file_path)
        elif file_path.endswith('.pdf'):
            text = read_pdf_file(file_path)
        else:
            raise ValueError(f"Formato no soportado: {file_path}")

        chunks = split_into_chunks(text)
        insert_into_chromadb(collection, chunks, embedding_model)

In [9]:
file_paths: list[str] = [os.path.join(os.getcwd(), 'data', 'resena.txt'),
                         os.path.join(os.getcwd(), 'data', 'lost-ruins-of-arnak-rules-es.pdf')]

In [14]:
process_and_store(file_paths)

Insert of existing embedding ID: doc_0
Add of existing embedding ID: doc_0
Insert of existing embedding ID: doc_1
Add of existing embedding ID: doc_1
Insert of existing embedding ID: doc_2
Add of existing embedding ID: doc_2
Insert of existing embedding ID: doc_3
Add of existing embedding ID: doc_3
Insert of existing embedding ID: doc_4
Add of existing embedding ID: doc_4
Insert of existing embedding ID: doc_5
Add of existing embedding ID: doc_5
Insert of existing embedding ID: doc_6
Add of existing embedding ID: doc_6
